In [1]:
! pip3 install --upgrade --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 49.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
PROJECT_ID = "qwiklabs-gcp-03-5cf9dc65558c"
LOCATION = "us-central1"

In [2]:
import os

os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["LOCATION"] = LOCATION
os.environ["API_ENDPOINT"] = f"{LOCATION}-aiplatform.googleapis.com"

In [3]:
os.environ["MODEL_ID"] = "gemini-1.5-pro"

## Generate content
The generateContent method can handle a wide variety of use cases, including multi-turn chat and multimodal input, depending on what the underlying model supports.

In [4]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:generateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": { "text": "Why is the sky blue?" }
    }
  }'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2989    0  2888  100   101    454     15  0:00:06  0:00:06 --:--:--   711


{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": "The sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's a simplified explanation:\n\n1. **Sunlight Enters the Atmosphere:** Sunlight, appearing white, is actually a mix of all colors. When this light enters the Earth's atmosphere, it encounters various particles like nitrogen and oxygen molecules.\n\n2. **Scattering of Light:** These particles are much smaller than the wavelength of visible light. When sunlight hits them, it gets scattered in different directions. This scattering, known as Rayleigh scattering, is more effective at shorter wavelengths, like blue and violet.\n\n3. **Blue Light Dominates:** While violet light has the shortest wavelength and scatters the most, our eyes are less sensitive to it. Blue light, being the next shortest, scatters almost as much and is more readily perceived by our eyes. \n\n4. **The Sky Appears Blue:** 

## Streaming
Send a text prompt to the model. The Gemini Pro (gemini-pro) model provides a streaming response mechanism. With this approach, we don't need to wait for the complete response; we can start processing fragments as soon as they're accessible.

In [5]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:streamGenerateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": { "text": "Why is the sky blue?" }
    }
  }'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


[{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": "The"
          }
        ]
      }
    }
  ],
  "usageMetadata": {},
  "modelVersion": "gemini-1.5-pro-001"
}
,
{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": " sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's"
          }
        ]
      },
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.08886719,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.026000977
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.23925781,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.103515625
        },
        {
         

100 13020    0 12919  100   101   1936     15  0:00:06  0:00:06 --:--:--  1722


          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.13476563,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.03515625
        },
        {
          "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.140625,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.024780273
        }
      ]
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 6,
    "candidatesTokenCount": 325,
    "totalTokenCount": 331
  },
  "modelVersion": "gemini-1.5-pro-001"
}
]

## Model parameters
Every prompt you send to the model includes parameter values that control how the model generates a response. The model can generate different results for different parameter values. You can experiment with different model parameters to see how the results change.

In [6]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:generateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": [
        {"text": "Describe this image"},
        {"file_data": {
          "mime_type": "image/png",
          "file_uri": "gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg"
        }}
      ]
    },
    "generation_config": {
      "temperature": 0.2,
      "top_p": 0.1,
      "top_k": 16,
      "max_output_tokens": 2048,
      "candidate_count": 1,
      "stop_sequences": []
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2301    0  1698  100   603    851    302  0:00:01  0:00:01 --:--:--  1153


{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": "A brown tabby cat is carefully walking across a snow-covered surface. The cat has bright yellow eyes and a slightly concerned expression. Its tail is held high, and its fur looks thick and warm against the cold snow. The background is blurred, emphasizing the cat as the focus of the image. \n"
          }
        ]
      },
      "finishReason": "STOP",
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.018798828,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.055908203
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.057373047,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.057373047
        },
     

## Chat
The Gemini Pro model supports natural multi-turn conversations and is ideal for text tasks that require back-and-forth interactions.

We should specify the role field only if the content represents a turn in a conversation. You can set role to one of the following values: user, model.

In [7]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:generateContent \
  -d '{
    "contents": [
      {
        "role": "user",
        "parts": [
          { "text": "Hello" }
        ]
      },
      {
        "role": "model",
        "parts": [
          { "text": "Hello! I am glad you could both make it." }
        ]
      },
      {
        "role": "user",
        "parts": [
          { "text": "So what is the first order of business?" }
        ]
      }
    ]
  }'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1978    0  1580  100   398   1083    272  0:00:01  0:00:01 --:--:--  1356


{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": "That depends! I'm ready for anything - to chat, answer questions, write stories, translate languages, or anything else you can think of.  \n\nWhat's on your mind today? 😊 \n"
          }
        ]
      },
      "finishReason": "STOP",
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.024780273,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.014526367
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.059326172,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.06542969
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore

## Function calling
Function calling lets you create a description of a function in their code, then pass that description to a language model in a request. This is an example of passing in a description of a function that returns information about where a movie is playing. Several function declarations are included in the request, such as find_movies and find_theaters.

In [8]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1beta1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:generateContent \
  -d '{
  "contents": {
    "role": "user",
    "parts": {
      "text": "Which theaters in Mountain View show Barbie movie?"
    }
  },
  "tools": [
    {
      "function_declarations": [
        {
          "name": "find_movies",
          "description": "find movie titles currently playing in theaters based on any description, genre, title words, etc.",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
              },
              "description": {
                "type": "string",
                "description": "Any kind of description including category or genre, title words, attributes, etc."
              }
            },
            "required": [
              "description"
            ]
          }
        },
        {
          "name": "find_theaters",
          "description": "find theaters based on location and optionally movie title which are is currently playing in theaters",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
              },
              "movie": {
                "type": "string",
                "description": "Any movie title"
              }
            },
            "required": [
              "location"
            ]
          }
        },
        {
          "name": "get_showtimes",
          "description": "Find the start times for movies playing in a specific theater",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
              },
              "movie": {
                "type": "string",
                "description": "Any movie title"
              },
              "theater": {
                "type": "string",
                "description": "Name of theater"
              },
              "date": {
                "type": "string",
                "description": "Date for requested showtime"
              }
            },
            "required": [
              "location",
              "movie",
              "theater",
              "date"
            ]
          }
        }
      ]
    }
  ]
}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4150    0  1587  100  2563   1016   1641  0:00:01  0:00:01 --:--:--  2658


{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "functionCall": {
              "name": "find_theaters",
              "args": {
                "location": "Mountain View, CA",
                "movie": "Barbie"
              }
            }
          }
        ]
      },
      "finishReason": "STOP",
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.15722656,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.12158203
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.13671875,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.17871094
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE",
          "probabili

## Use the Gemini Pro Vision Model
The Gemini Pro Vision (gemini-pro-vision) is a multimodal model that supports adding image and video in text or chat prompts for a text response.

In [9]:
! gsutil cp "gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg" ./image.jpg

Copying gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg...
/ [1 files][ 17.4 KiB/ 17.4 KiB]                                                
Operation completed over 1 objects/17.4 KiB.                                     


## Generate text from a local image
Specify the base64 encoding of the image or video to include inline in the prompt and the mime_type field. The supported MIME types for images include image/png and image/jpeg.

In [10]:
%%bash

# Encode image data in base64
# NOTE: This command only works on Linux.
data=$(base64 -w 0 image.jpg)

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:generateContent \
  -d "{
      'contents': {
        'role': 'USER',
        'parts': [
          {
            'text': 'Is it a cat?'
          },
          {
            'inline_data': {
              'data': '${data}',
              'mime_type':'image/jpeg'
            }
          }
        ]
      }
    }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25606    0  1514  100 24092   1196  19045  0:00:01  0:00:01 --:--:-- 20241


{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": "Yes, the picture is of a cat.  More specifically, it appears to be a tabby cat based on its striped pattern. \n"
          }
        ]
      },
      "finishReason": "STOP",
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.04345703,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.09277344
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.060058594,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.08251953
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.08251953,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",

## Generate text from an image on Google Cloud Storage
Specify the Cloud Storage URI of the image to include in the prompt. The bucket that stores the file must be in the same Google Cloud project that's sending the request. You must also specify the mime_type field. The supported image MIME types include image/png and image/jpeg.

In [11]:
%%bash

MODEL_ID="gemini-1.5-pro"

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:generateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": [
        {
          "text": "Describe this image"
        },
        {
          "file_data": {
            "mime_type": "image/png",
            "file_uri": "gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg"
          }
        }
      ]
    },
    "generation_config": {
      "temperature": 0.2,
      "top_p": 0.1,
      "top_k": 16,
      "max_output_tokens": 2048,
      "candidate_count": 1,
      "stop_sequences": []
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2492    0  1843  100   649    705    248  0:00:02  0:00:02 --:--:--   953


{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": "A brown tabby cat with bright yellow eyes is cautiously walking across a snow-covered ground. The cat's fur is a mix of brown, black, and gray stripes, and its tail is bushy and held low. The cat's ears are perked up, and it appears to be alert and focused on something in the distance. The background is a snowy landscape, with tire tracks visible in the snow. The sunlight reflects off the snow, creating a bright and wintry atmosphere. \n"
          }
        ]
      },
      "finishReason": "STOP",
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.017456055,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.05493164
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
 